In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

import os
import pandas as pd
from datetime import datetime, timedelta

In [2]:

#Code Pikeln
def pickeln_data_from_wikidata():
    cache_file = 'data_cache.csv'
    cache_duration = timedelta(days=7)  # Daten alle 7 Tage aktualisieren

    # Überprüfen, ob eine zwischengespeicherte Datei existiert und aktuell ist
    if os.path.exists(cache_file):
        file_mod_time = datetime.fromtimestamp(os.path.getmtime(cache_file))
        if datetime.now() - file_mod_time < cache_duration:
            return pd.read_csv(cache_file)

    # SPARQL-Abfrage an Wikidata
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery('''
        SELECT DISTINCT ?school ?label_school ?location ?label_location ?location_coordinates ?website
        WHERE {
          SERVICE <https://query.wikidata.org/sparql> {
            ?school wdt:P31 wd:Q15850083;     	# is a university of applied science
                    rdfs:label ?label_school;       	 
                    wdt:P625 ?location_coordinates;
                    wdt:P131 ?location;
                    wdt:P856 ?website.
            ?location rdfs:label ?label_location.



            OPTIONAL {?school wdt:P1813 ?shortname;   	 
                            wdt:P1366 ?school_replace.  	 
                    ?school_replace rdfs:label ?label_replace.}
            OPTIONAL {?location wdt:P625 ?location_coordinates.}

            FILTER (LANG(?label_school) = "en" &&      	# Englisch sind mehr verfügbar als in Deutsch
                    LANG(?label_location) = "en")
            }
          }
        ORDER BY ASC (?label_school)
    ''')
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    # Verarbeiten der Ergebnisse
    data = []
    for result in results["results"]["bindings"]:
        entry = {
            'school': result["school"]["value"],
            'label_school': result["label_school"]["value"],
            'location': result["location"]["value"],
            'label_location': result["label_location"]["value"],
            'website': result["website"]["value"]
        }
        if 'location_coordinates' in result:
            entry['location_coordinates'] = result["location_coordinates"]["value"]
        else:
            entry['location_coordinates'] = None

        data.append(entry)

    df = pd.DataFrame(data)
    # Speichern in einer CSV-Datei
    df.to_csv(cache_file, index=False)

    return df

In [4]:
data1 = pickeln_data_from_wikidata()
data1.head(10)

,school,label_school,location,label_location,website,location_coordinates
0,http://www.wikidata.org/entity/Q1622181,Academy of Art and Design,http://www.wikidata.org/entity/Q78,Basel,http://www.fhnw.ch/hgk/,Point(7.611 47.533055555)
1,http://www.wikidata.org/entity/Q1622152,Bern Academy of Arts,http://www.wikidata.org/entity/Q70,Bern,http://www.hkb.bfh.ch/,Point(7.391388888 46.946666666)
2,http://www.wikidata.org/entity/Q466455,Bern University of Applied Sciences,http://www.wikidata.org/entity/Q11911,Bern (canton),https://www.bfh.ch,Point(7.277219444 47.1583)
3,http://www.wikidata.org/entity/Q98830957,Business Department,http://www.wikidata.org/entity/Q70,Bern,https://www.bfh.ch/wirtschaft,Point(7.4403856 46.9423434)
4,http://www.wikidata.org/entity/Q98830957,Business Department,http://www.wikidata.org/entity/Q11911,Bern (canton),https://www.bfh.ch/wirtschaft,Point(7.4403856 46.9423434)
5,http://www.wikidata.org/entity/Q98830957,Business Department,http://www.wikidata.org/entity/Q514089,Mattenhof-Weissenbühl,https://www.bfh.ch/wirtschaft,Point(7.4403856 46.9423434)
6,http://www.wikidata.org/entity/Q98830957,Business Department,http://www.wikidata.org/entity/Q666217,Bern-Mittelland administrative district,https://www.bfh.ch/wirtschaft,Point(7.4403856 46.9423434)
7,http://www.wikidata.org/entity/Q98830957,Business Department,http://www.wikidata.org/entity/Q1625002,Sandrain,https://www.bfh.ch/wirtschaft,Point(7.4403856 46.9423434)
8,http://www.wikidata.org/entity/Q691146,City of Basel Music Academy,http://www.wikidata.org/entity/Q78,Basel,http://www.musik-akademie.ch,Point(7.5861 47.5552)
9,http://www.wikidata.org/entity/Q1622221,Fachhochschule Nordwestschweiz. Hochschule für...,http://www.wikidata.org/entity/Q66191,Muttenz,http://www.fhnw.ch/technik/,Point(8.212903 47.479639)
